In [ ]:
!pip install -r requirements.txt
!python -m spacy download en_core_web_sm

In [ ]:
from concurrent.futures import ProcessPoolExecutor
from functools import partial
import os
import random
import time
import warnings
import zipfile
from tqdm import tqdm
import json

import evaluate
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import parquet
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertConfig, BertTokenizer
from unidecode import unidecode

from src.features.functions_preprocessing import (
    plot_text_length_distribution,
    preprocess_articles,
    preprocess_summaries,
)
from src.features.tokenization import parallel_tokenize
from src.models.bert import BertSummary
from src.models.rnn_encoder_decoder import Encoder, Decoder, Seq2Seq
from src.models.transformer import Transformer
from src.models.train_models import train_model
from src.evaluation.model_evaluation import (
    generate_summaries_seq2seq,
    generate_summaries_transformer,
    generate_summaries_bert,
)

In [ ]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)
print(f"Using {device} device")

In [ ]:
def get_allowed_cpu_count():
    # Returns the number of CPU cores available for this process.
    try:
        return len(os.sched_getaffinity(0))
    except AttributeError:
        return os.cpu_count() or 1


cpu_count = get_allowed_cpu_count()
print(cpu_count)

In [ ]:
n_process = max(1, cpu_count // 2)

In [ ]:
torch.set_num_threads(n_process)

# **Kaggle dataset**

In [ ]:
!kaggle datasets download -d sbhatti/news-summarization

In [ ]:
with zipfile.ZipFile("news-summarization.zip", "r") as zip_ref:
    zip_ref.extractall("news-summarization")

In [ ]:
news_data = pd.read_csv("news-summarization/data.csv")

In [ ]:
news_data.head()

In [ ]:
N = random.randint(1, len(news_data))

print(news_data["Content"][N])
print()
print(news_data["Summary"][N])

In [ ]:
lengths_article = news_data["Content"].str.len()
lengths_article.describe()

In [ ]:
news_data = news_data[
    (lengths_article >= lengths_article.quantile(0.10))
    & (lengths_article <= lengths_article.quantile(0.90))
]

In [ ]:
plot_text_length_distribution(news_data, "Content")

In [ ]:
lengths_summary = news_data["Summary"].str.len()
lengths_summary.describe()

In [ ]:
news_data = news_data[
    (lengths_summary >= lengths_summary.quantile(0.10))
    & (lengths_summary <= lengths_summary.quantile(0.90))
]

In [ ]:
news_data["Summary"].str.len().describe()

In [ ]:
plot_text_length_distribution(news_data, "Summary")

In [ ]:
len(news_data)

In [ ]:
# news_data.loc[:, "Content"] = preprocess_articles(
#     news_data["Content"].tolist(), n_process=n_process, batch_size=32
# )
# news_data.loc[:, "Summary"] = preprocess_summaries(
#     news_data["Summary"].tolist(), n_process=n_process, batch_size=32
# )

In [ ]:
# news_data.to_parquet("news_data_cleaned.parquet", index=False)

In [ ]:
news_data = pd.read_parquet("news_data_cleaned.parquet")

# **Tokenization**

In [ ]:
data_copy = news_data[:]
data_copy = news_data.sample(frac=1, random_state=42)

train_ratio = 0.8
train_size = int(train_ratio * len(data_copy))

# Slice the dataset
train_data = data_copy[:train_size]
test_data = data_copy[train_size:]

print(f"Train size: {len(train_data)}")
print(f"Test size:  {len(test_data)}")

In [ ]:
if __name__ == "__main__":
    texts_content = list(train_data["Content"])
    print("Tokenizing Content...")
    tokenized_articles = parallel_tokenize(
        texts_content,
        tokenizer_name="bert-base-uncased",
        max_workers=n_process,
        chunk_size=2000,
        max_length=512,
    )
    print("tokenized_articles.shape =", tokenized_articles.shape)
    torch.save(tokenized_articles, "tokenized_articles.pt")

In [ ]:
if __name__ == "__main__":
    texts_summary = list(train_data["Summary"])
    print("Tokenizing Summaries...")
    tokenized_summaries = parallel_tokenize(
        texts_summary,
        tokenizer_name="bert-base-uncased",
        max_workers=n_process,
        chunk_size=2000,
        max_length=129,
    )
    print("tokenized_summaries.shape =", tokenized_summaries.shape)
    torch.save(tokenized_summaries, "tokenized_summaries.pt")

In [ ]:
if __name__ == "__main__":
    texts_content = list(test_data["Content"])
    print("Tokenizing Content...")
    tokenized_articles_test = parallel_tokenize(
        texts_content,
        tokenizer_name="bert-base-uncased",
        max_workers=n_process,
        chunk_size=2000,
        max_length=512,
    )
    print("tokenized_articles.shape =", tokenized_articles_test.shape)
    torch.save(tokenized_articles_test, "tokenized_articles_test.pt")

In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    tokenized_articles = torch.load("tokenized_articles.pt")
    tokenized_summaries = torch.load("tokenized_summaries.pt")
    tokenized_summaries_bert = torch.load("tokenized_summaries_bert.pt")
    tokenized_articles_test = torch.load("tokenized_articles_test.pt")

In [ ]:
article_ids = tokenized_articles.long()
summary_ids = tokenized_summaries.long()

# **Seq2Seq**

In [ ]:
batch_size = 32

dataset = TensorDataset(tokenized_articles, tokenized_summaries)
dataloader = DataLoader(
    dataset, batch_size=batch_size, num_workers=n_process, shuffle=True
)

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
encoder = Encoder(
    vocab_size=tokenizer.vocab_size,
    embed_dim=128,
    hidden_size=128,
    num_layers=2,
    dropout_prob=0.1,
)
decoder = Decoder(
    vocab_size=tokenizer.vocab_size,
    embed_dim=128,
    hidden_size=128,
    num_layers=2,
    dropout_prob=0.1,
)

modelSeq2Seq = Seq2Seq(encoder, decoder, device).to(device)

In [ ]:
train_model(
    model=modelSeq2Seq,
    dataloader=dataloader,
    num_epochs=30,
    optimizer=torch.optim.Adam(modelSeq2Seq.parameters(), lr=2e-4),
    loss_fn=nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id),
    model_name="Seq2Seq",
    device=device,
    teacher_forcing_ratio=0.5,
)

In [ ]:
encoder = Encoder(
    vocab_size=tokenizer.vocab_size,
    embed_dim=128,
    hidden_size=128,
    num_layers=2,
    dropout_prob=0.1,
)
decoder = Decoder(
    vocab_size=tokenizer.vocab_size,
    embed_dim=128,
    hidden_size=128,
    num_layers=2,
    dropout_prob=0.1,
)

modelSeq2Seq = Seq2Seq(encoder, decoder, device).to(device)

modelSeq2Seq.load_state_dict(torch.load("model_weights/seq2seq_weights_25_epochs.pth"))
modelSeq2Seq.eval()

# **Transformer**

In [ ]:
batch_size = 32

dataset = TensorDataset(tokenized_articles, tokenized_summaries)
dataloader = DataLoader(
    dataset, batch_size=batch_size, num_workers=n_process, shuffle=True
)

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
modelTransformer = Transformer(
    pad_idx=0,
    voc_size=tokenizer.vocab_size,
    hidden_size=128,
    n_head=8,
    max_len=512,
    dec_max_len=512,
    ffn_hidden=128,
    n_layers=3,
)

In [ ]:
train_model(
    model=modelTransformer,
    dataloader=dataloader,
    num_epochs=25,
    optimizer=torch.optim.Adam(modelTransformer.parameters(), lr=2e-4),
    loss_fn=nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id),
    model_name="Transformer",
    device=device,
)

In [ ]:
modelTransformer = Transformer(
    pad_idx=0,
    voc_size=tokenizer.vocab_size,
    hidden_size=128,
    n_head=8,
    max_len=512,
    dec_max_len=128,
    ffn_hidden=128,
    n_layers=3,
)
modelTransformer.load_state_dict(
    torch.load("model_weights/transformer_weights_25_epochs.pth")
)
modelTransformer.eval()

# **BERT model**

In [ ]:
batch_size = 8

dataset = TensorDataset(tokenized_articles, tokenized_summaries[:, 1:])
dataloader = DataLoader(
    dataset, batch_size=batch_size, num_workers=n_process, shuffle=True
)

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
config = BertConfig(
    hidden_size=768,
    num_hidden_layers=12,
    num_attention_heads=12,
    intermediate_size=3072,
    hidden_act="gelu",
    hidden_dropout_prob=0.1,
    attention_probs_dropout_prob=0.1,
    max_position_embeddings=512,
    vocab_size=tokenizer.vocab_size,
)

In [ ]:
modelBert = BertSummary(config)

In [ ]:
train_model(
    model=modelBert,
    dataloader=dataloader,
    num_epochs=10,
    optimizer=torch.optim.Adam(modelBert.parameters(), lr=1e-5),
    loss_fn=nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id),
    model_name="BERT",
    device=device,
)

In [ ]:
modelBert = BertSummary(config)
modelBert.load_state_dict(torch.load("model_weights/bert_weights_7_epochs.pth"))
modelBert.to(device)
modelBert.eval()

# **Evaluation**

In [ ]:
rouge = evaluate.load("rouge")

In [ ]:
input_text = news_data["Content"][1000]
summary = news_data["Summary"][1000]
print(summary)

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
tokenized_input = parallel_tokenize(
    input_text,
    tokenizer_name="bert-base-uncased",
    max_workers=n_process,
    chunk_size=2000,
    max_length=512,
)

## Seq2Seq

In [ ]:
predictions_seq2seq = generate_summaries_seq2seq(
    model=modelSeq2Seq,
    batch_size=32,
    tokenized_input=tokenized_articles_test,
    limit=10,
)

with open("predictions_seq2seq.json", "w") as f:
    json.dump(predictions_seq2seq, f)

In [ ]:
with open("predictions_seq2seq.json", "r") as f:
    predictions_seq2seq = json.load(f)

In [ ]:
reference_summaries = list(test_data["Summary"][:320])
results = rouge.compute(predictions=predictions_seq2seq, references=reference_summaries)
print("ROUGE metrics:", results)

## Transformer

In [ ]:
predictions_transformer = generate_summaries_transformer(
    model=modelTransformer,
    batch_size=32,
    tokenized_input=tokenized_articles_test,
    limit=10,
)

with open("predictions_transformer.json", "w") as f:
    json.dump(predictions_transformer, f)

In [ ]:
with open("predictions_transformer.json", "r") as f:
    predictions_transformer = json.load(f)

In [ ]:
reference_summaries = list(test_data["Summary"][:320])
results = rouge.compute(
    predictions=predictions_transformer, references=reference_summaries
)
print("ROUGE metrics:", results)

## Bert

In [ ]:
predictions_bert = generate_summaries_bert(
    model=modelBert,
    batch_size=32,
    tokenized_input=tokenized_articles_test,
    limit=10,
)

with open("predictions_bert.json", "w") as f:
    json.dump(predictions_bert, f)

In [ ]:
with open("predictions_bert.json", "r") as f:
    predictions_bert = json.load(f)

In [ ]:
reference_summaries = list(test_data["Summary"][:320])
results = rouge.compute(predictions=predictions_bert, references=reference_summaries)
print("ROUGE metrics:", results)